# Домашнее задание № 3. Исправление опечаток

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандидатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [1]:
import textdistance
import re
import numpy as np
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

Немного изменим предобработку корпуса, убрав из него символы ударения и добавив в токенизацию слова с дефисом

In [2]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
corpus = re.sub(chr(769), '', corpus)
vocab = Counter(re.findall('\w+-\w+|\w+', corpus.lower()))
N = sum(vocab.values())

Добавим все необходимые функции из ноутбука с пары. Несколько изменим функцию `get_closest_match_with_metric`: добавим после метрики расчет вероятности слова для более удобной дальнейшей фильтрации.

Для сравнения исправления опечаток скопируем функцию `get_closest_hybrid_match` в `pr_get_closest_hybrid_match`.

В функции `get_closest_hybrid_match` добавим отбор кандидатов по вероятности следующим образом:

    пройдемся по списку с полученными метриками,
    добавим в новый список кандидата с его метрикой и вероятностью, если
        в списке нет слова с такой же метрикой И его вероятность выше, чем у слова с такой же метрикой кандидата в списке

In [15]:
def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_match_with_metric(text, lookup, topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = (metric.normalized_similarity(text, word), P(word))
    
    return similarities.most_common(topn)

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]
 
def pr_get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    return closest

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest_metric_pr = get_closest_match_with_metric(text, lookup, topn, metric=metric)
    
    closest = []
    for w, m in closest_metric_pr:
        if closest:
            if m[0] == closest[-1][1][0]:
                if m[1] > closest[-1][1][1]:
                    closest.pop()
                    closest.append((w, m))
            else:
                closest.append((w, m))
        else:
            closest.append((w, m))
    
    return closest

In [4]:
word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [5]:
get_closest_match_with_metric('шде-то', vocab, topn=5)

[('где-то', (0.8333333333333334, 1.0466768503716196e-05)),
 ('чье-то', (0.6666666666666667, 3.949723963666489e-07)),
 ('детто', (0.6666666666666667, 1.9748619818332446e-07)),
 ('де-чо', (0.6666666666666667, 1.9748619818332446e-07)),
 ('удето', (0.6666666666666667, 1.9748619818332446e-07))]

In [14]:
get_closest_hybrid_match('колмпания', X, vec)

[('компания', (0.8888888888888888, 0.0001840571367068584)),
 ('компаниям', (0.7777777777777778, 4.542182558216463e-06)),
 ('кинокомпания', (0.6666666666666667, 5.924585945499734e-07))]

Пример применения функции для слова с несколькими кандидатами с одинаковой метрикой

In [17]:
pr_get_closest_hybrid_match('ромашк', X, vec)

[('ромашка', (0.8571428571428572, 2.172348180016569e-06)),
 ('ромашки', (0.8571428571428572, 3.949723963666489e-07)),
 ('ромашке', (0.8571428571428572, 1.9748619818332446e-07))]

In [18]:
get_closest_hybrid_match('ромашк', X, vec)

[('ромашка', (0.8571428571428572, 2.172348180016569e-06))]

In [19]:
3.949723963666489e-07 < 2.172348180016569e-06

True

## 2.  Symspell (5 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Там к словам применяется только одна операция - удаление символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово   
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [20]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()
corpus = re.sub(chr(769), '', corpus)
vocab = Counter(re.findall('\w+-\w+|\w+', corpus.lower()))
N = sum(vocab.values())

Заводим функцию, которая генерирует удаления (позаимствована из ноутбука с пары)

Добавляем в словарь удалений только не пустые строчки (потому что удаление, например, для предлога *в* - ""). Также убираем лишние знаки пунктуации, если попадаются, и цифры

In [21]:
def generate_deletes(word):    
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    return [L + R[1:] for L, R in splits if R]

deletes_vocab = {}
for v in vocab:
    for w in generate_deletes(v):
        if w and not w.isdigit():
            if not w in deletes_vocab:
                deletes_vocab[w] = [v]
            else:
                deletes_vocab[w].append(v)

Получившийся словарь удалений:

{удаление: варианты исправления списком}

In [22]:
deletes_vocab

{'овостройка': ['новостройка'],
 'нвостройка': ['новостройка'],
 'ноостройка': ['новостройка'],
 'новстройка': ['новостройка'],
 'новотройка': ['новостройка'],
 'новосройка': ['новостройка'],
 'новостойка': ['новостройка'],
 'новострйка': ['новостройка'],
 'новострока': ['новостройка'],
 'новостройа': ['новостройка'],
 'новостройк': ['новостройка', 'новостройки'],
 'ижегородская': ['нижегородская'],
 'нжегородская': ['нижегородская'],
 'ниегородская': ['нижегородская'],
 'нижгородская': ['нижегородская'],
 'нижеородская': ['нижегородская'],
 'нижегродская': ['нижегородская'],
 'нижегоодская': ['нижегородская'],
 'нижегордская': ['нижегородская'],
 'нижегороская': ['нижегородская'],
 'нижегородкая': ['нижегородская'],
 'нижегородсая': ['нижегородская'],
 'нижегородскя': ['нижегородская'],
 'нижегородска': ['нижегородская'],
 'бласть': ['область'],
 'оласть': ['область'],
 'обасть': ['область'],
 'облсть': ['область'],
 'облать': ['область'],
 'облась': ['область'],
 'област': ['область'

Пишем функцию по исправлению опечаток

* генерируем удаления,

* добавляем в список кандидатов а) варианты исправления текущего удаления, б) само удаление, если оно есть в изначальном словаре,

* возвращаем
        - самого вероятного кандидата на исправление, если удалось найти хоть какие-то варианты исправления,
        - изначальное слово во всех других случаях

In [59]:
def P(word, N=N):
    return vocab[word] / N

def correction(vocab, word: 'str') -> str:
    w_deletes = generate_deletes(word)
    d_candidates = []
    for w in w_deletes:
        if w in deletes_vocab:
            d_candidates.extend(deletes_vocab.get(w))
        elif w in vocab:
            d_candidates.append(w)
    if d_candidates:
        return max(d_candidates, key=P)
    return word

In [60]:
%%time
correction(vocab, 'прогрраммирование')

Wall time: 0 ns


'программированием'

In [61]:
%%time
correction(vocab, 'лекция')

Wall time: 0 ns


'лекции'

In [26]:
from tqdm.notebook import tqdm

Попробуем процентно оценить качество работы функции на данных

In [27]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [28]:
from string import punctuation

def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

In [63]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        # чтобы два раза не исправлять одно и тоже слово - закешируем его
        # перед тем как считать исправление проверим нет ли его в кеше
        
        predicted = cashed.get(pair[1], correction(vocab, pair[1]))
        cashed[pair[1]] = predicted
        
        if predicted == pair[0]:
            correct += 1
        total += 1
        
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted:
                mistaken_fixed += 1

  0%|          | 0/915 [00:00<?, ?it/s]

In [64]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.5198599299649825
0.2857142857142857
0.44550361777879866


Только 28% опечаток исправляются корректно.

Посмотрим, как исправляются самые частотные опечатки

In [41]:
mistakes = []
total = 0
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        total += 1

In [58]:
[(wt[0], wt[1], correction(vocab, wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'седан'),
 ('вообще', 'вобще', 'общей'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естественной'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'статей'),
 ('очень', 'ооочень', ''),
 ('как-то', 'както', 'какой'),
 ('очень', 'оооочень', ''),
 ('это', 'ето', 'что')]

## *3. Чтение. (2 балла)

Прочитайте эту главу в книге Speech and Language Processing - https://web.stanford.edu/~jurafsky/slp3/2.pdf .
Ответьте на следующий вопрос:

1. Что такое Byte-Pair Encoding (опишите по-русски, как минимум 10 предложениями)?

*Это задание не связано напрямую с исправлением опечаток, но это важная тема, к которой мы вернемся в конце курса

1. Один из методов токенизации, который выучивает некоторые подпоследовательности из строк, а затем применяет их при сегментации сырого текста. Т. е. по идее он должен находить в тексте некоторые морфемы вроде частотных суффиксов и приставок. 
2. Самый простой из алгоритмов токенизации такого рода. SentencePiece частично включает в себя именно этот алгоритм.
3. Работает следующим образом: делит простейшие токены (разделенные пробелом) в корпусе посимвольно, находит самые частотные пары символов и объединяет их в одну последовательность. Эту последовательность добавляет в словарь символов. И так далее до некоторого критерия остановки.

Из интереса решила попробовать написать немного кода на это, чтобы лучше понять. Оставлю ниже

In [126]:
bpe_corpus = '''Once we’ve learned our vocabulary, the token parser is used to tokenize a test
sentence. The token parser just runs on the test data the merges we have learned
from the training data, greedily, in the order we learned them. (Thus the frequencies
in the test data don’t play a role, just the frequencies in the training data). So first
we segment each test sentence word into characters. Then we apply the first rule:
replace every instance of er in the test corpus with er, and then the second rule:
replace every instance of er in the test corpus with er , and so on. By the end,
if the test corpus contained the word newer, it would be tokenized as a full
word. But a new (unknown) word like lower would be merged into the two
tokens low er.'''

In [127]:
bpe_corpus_l = [token.strip(punctuation) for token in bpe_corpus.lower().split()]

In [128]:
corpus_vocab = Counter(bpe_corpus_l)

In [129]:
corpus_vocab

Counter({'once': 1,
         'we’ve': 1,
         'learned': 3,
         'our': 1,
         'vocabulary': 1,
         'the': 18,
         'token': 2,
         'parser': 2,
         'is': 1,
         'used': 1,
         'to': 1,
         'tokenize': 1,
         'a': 4,
         'test': 7,
         'sentence': 2,
         'just': 2,
         'runs': 1,
         'on': 2,
         'data': 4,
         'merges': 1,
         'we': 4,
         'have': 1,
         'from': 1,
         'training': 2,
         'greedily': 1,
         'in': 5,
         'order': 1,
         'them': 1,
         'thus': 1,
         'frequencies': 2,
         'don’t': 1,
         'play': 1,
         'role': 1,
         'so': 2,
         'first': 2,
         'segment': 1,
         'each': 1,
         'word': 4,
         'into': 2,
         'characters': 1,
         'then': 2,
         'apply': 1,
         'rule': 2,
         'replace': 2,
         'every': 2,
         'instance': 2,
         'of': 2,
         'er': 5,
 

In [130]:
v_to_freq = [(corpus_vocab[k], list(k) + ['_']) for k in corpus_vocab]

In [131]:
v_to_freq[0]

(1, ['o', 'n', 'c', 'e', '_'])

In [132]:
def initialize_vocabulary(v_to_freq, vocabulary):
    for fr, syms in v_to_freq:
        for s in syms:
            if not s in vocabulary:
                vocabulary.append(s)
    return vocabulary
                
def find_frequent_pairs(v_to_freq):
    by_counter = Counter()
    for fr, syms in v_to_freq:
        bygrams = [tuple(syms[i:i+2]) for i in range(len(syms) - 1)]
        for b in bygrams:
            by_counter[b] += fr
    return by_counter.most_common(1)[0]

def merge_most_common(v_to_freq, mc_pair):
    new_sym = ''.join(mc_pair[0])
    new_v_to_freq = []
    for fr, syms in v_to_freq:
        if mc_pair[0] in [tuple(syms[i:i+2]) for i in range(len(syms) - 1)]:
            sym_parts = ''.join(syms).split(new_sym)
            syms_new = []
            for i in range(len(sym_parts) - 1):
                syms_new.extend(sym_parts[i])
                syms_new.append(new_sym)
            syms_new.extend(sym_parts[-1])
            new_v_to_freq.append((fr, syms_new))
        else:
            new_v_to_freq.append((fr, syms))
    return new_v_to_freq

In [133]:
bc = find_frequent_pairs(v_to_freq)

In [135]:
bc

(('e', '_'), 38)

In [136]:
res = merge_most_common(v_to_freq, bc)

In [137]:
res

[(1, ['o', 'n', 'c', 'e_']),
 (1, ['w', 'e', '’', 'v', 'e_']),
 (3, ['l', 'e', 'a', 'r', 'n', 'e', 'd', '_']),
 (1, ['o', 'u', 'r', '_']),
 (1, ['v', 'o', 'c', 'a', 'b', 'u', 'l', 'a', 'r', 'y', '_']),
 (18, ['t', 'h', 'e_']),
 (2, ['t', 'o', 'k', 'e', 'n', '_']),
 (2, ['p', 'a', 'r', 's', 'e', 'r', '_']),
 (1, ['i', 's', '_']),
 (1, ['u', 's', 'e', 'd', '_']),
 (1, ['t', 'o', '_']),
 (1, ['t', 'o', 'k', 'e', 'n', 'i', 'z', 'e_']),
 (4, ['a', '_']),
 (7, ['t', 'e', 's', 't', '_']),
 (2, ['s', 'e', 'n', 't', 'e', 'n', 'c', 'e_']),
 (2, ['j', 'u', 's', 't', '_']),
 (1, ['r', 'u', 'n', 's', '_']),
 (2, ['o', 'n', '_']),
 (4, ['d', 'a', 't', 'a', '_']),
 (1, ['m', 'e', 'r', 'g', 'e', 's', '_']),
 (4, ['w', 'e_']),
 (1, ['h', 'a', 'v', 'e_']),
 (1, ['f', 'r', 'o', 'm', '_']),
 (2, ['t', 'r', 'a', 'i', 'n', 'i', 'n', 'g', '_']),
 (1, ['g', 'r', 'e', 'e', 'd', 'i', 'l', 'y', '_']),
 (5, ['i', 'n', '_']),
 (1, ['o', 'r', 'd', 'e', 'r', '_']),
 (1, ['t', 'h', 'e', 'm', '_']),
 (1, ['t', 'h', 'u

In [142]:
vocab = initialize_vocabulary(v_to_freq, [])
used_vtf = v_to_freq.copy()

for i in range(25):
    next_pair = find_frequent_pairs(used_vtf)
    vocab.append(''.join(next_pair[0]))
    used_vtf = merge_most_common(used_vtf, next_pair)

In [145]:
used_vtf != v_to_freq

True

In [144]:
vocab

['o',
 'n',
 'c',
 'e',
 '_',
 'w',
 '’',
 'v',
 'l',
 'a',
 'r',
 'd',
 'u',
 'b',
 'y',
 't',
 'h',
 'k',
 'p',
 's',
 'i',
 'z',
 'j',
 'm',
 'g',
 'f',
 'q',
 'e_',
 'th',
 'the',
 'the_',
 'd_',
 'er',
 't_',
 'in',
 'en',
 's_',
 'st_',
 'er_',
 'to',
 'a_',
 'te',
 'st',
 'or',
 'd_',
 'en',
 'ar',
 'te',
 't_',
 'es',
 'y_',
 'er']

**Вывод:** получилось любопытно. За 25 итераций даже нашелся определенный артикль